In [1]:
# load packages
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

**County Population**

In [2]:
# read in county population files from three separate csvs
cplist = []

for i in ['2009', '2019', '2021']:
    df = pd.read_csv(f'data/county_pop/co-est{i}-alldata.csv', header=0, encoding='latin1')
    cplist.append(df)

#cplist[0]['CENSUS2000POP'].dtype == cplist[1]['CENSUS2010POP'].dtype
#print(cplist[0]['CENSUS2000POP'].dtype)
#print(cplist[1]['CENSUS2010POP'].dtype)
#for i in range(0, 3):
#    print(cplist[i].dtypes)

In [3]:
#concatenate three dataframe in cplist
cp = pd.concat(cplist, axis=0, ignore_index=True)

#filter out state & county level summary rows
cp = cp[cp['CTYNAME'].str.contains('County|Parish|Borough')]

#pivot dataset from wide to long format
cp = cp.loc[:, [col for col in cp.columns if col not in ['CENSUS2000POP', 'CENSUS2010POP']]]
stub = list(cp.columns[cp.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())
cp["id"] = cp.index
cp = pd.wide_to_long(cp, stub, i = "id", j = "year").fillna(0).reset_index()

#group data by year and county
cp = cp.groupby(['year', 'COUNTY', 'STNAME', 'SUMLEV', 'REGION', 'DIVISION', 'STATE', 'CTYNAME']).agg(sum).reset_index()

#delete unneeded columns
deletecols = ['SUMLEV', 'REGION', 'DIVISION', 'id', 'GQESTIMATESBASE', 'GQESTIMATES', 'RBIRTH', 'RDEATH', 'RNATURALINC', 'RINTERNATIONALMIG',\
               'RDOMESTICMIG', 'RNETMIG', 'NPOPCHG', 'NATURALCHG', 'RNATURALCHG']
cp = cp.loc[:, [col for col in cp.columns if col not in deletecols]]
print(list(cp.columns))

#cp[cp['CTYNAME'] == 'Miami-Dade County'].to_csv('data/county_pop/miami-dade-test.csv')

#convert FIPS codes to string and add dropped leading zeroes
cp[['STATE', 'COUNTY']] = cp[['STATE', 'COUNTY']].astype('string')
cp.loc[cp['STATE'].str.len() == 1, 'STATE'] = ['0' + i for i in cp.loc[cp['STATE'].str.len() == 1, 'STATE']]
cp.loc[cp['COUNTY'].str.len() == 1, 'COUNTY'] = ['00' + i for i in cp.loc[cp['COUNTY'].str.len() == 1, 'COUNTY']]
cp.loc[cp['COUNTY'].str.len() == 2, 'COUNTY'] = ['0' + i for i in cp.loc[cp['COUNTY'].str.len() == 2, 'COUNTY']]

#combine state and county FIPS codes to get the five-digit FIPS
cp = cp.assign(FIPS = cp['STATE'] + cp['COUNTY'])


C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3807842804.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  stub = list(cp.columns[cp.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())


['year', 'COUNTY', 'STNAME', 'STATE', 'CTYNAME', 'ESTIMATESBASE', 'POPESTIMATE', 'NPOPCHG_', 'BIRTHS', 'DEATHS', 'NATURALINC', 'INTERNATIONALMIG', 'DOMESTICMIG', 'NETMIG', 'RESIDUAL']


In [4]:
#check for null values by column
print(cp.isnull().sum())

#view first 10 rows of dataset
display(cp.head(10))

#view summary statistics on numerical columns
display(cp.describe())

year                0
COUNTY              0
STNAME              0
STATE               0
CTYNAME             0
ESTIMATESBASE       0
POPESTIMATE         0
NPOPCHG_            0
BIRTHS              0
DEATHS              0
NATURALINC          0
INTERNATIONALMIG    0
DOMESTICMIG         0
NETMIG              0
RESIDUAL            0
FIPS                0
dtype: int64


,year,COUNTY,STNAME,STATE,CTYNAME,ESTIMATESBASE,POPESTIMATE,NPOPCHG_,BIRTHS,DEATHS,NATURALINC,INTERNATIONALMIG,DOMESTICMIG,NETMIG,RESIDUAL,FIPS
0,2000,001,Alabama,01,Autauga County,43671.0,43872.0,201.0,142.0,79.0,63.0,5.0,133.0,138.0,0.0,01001
1,2000,001,Arizona,04,Apache County,69423.0,69516.0,93.0,277.0,106.0,171.0,1.0,-82.0,-81.0,3.0,04001
2,2000,001,Arkansas,05,Arkansas County,20743.0,20672.0,-71.0,49.0,46.0,3.0,0.0,-72.0,-72.0,-2.0,05001
3,2000,001,California,06,Alameda County,1443744.0,1450220.0,6476.0,5481.0,2302.0,3179.0,3648.0,-462.0,3186.0,111.0,06001
4,2000,001,Colorado,08,Adams County,347985.0,350961.0,2976.0,1507.0,472.0,1035.0,709.0,1220.0,1929.0,12.0,08001
5,2000,001,Connecticut,09,Fairfield County,882567.0,884420.0,1853.0,3175.0,1631.0,1544.0,1718.0,-1444.0,274.0,35.0,09001
6,2000,001,Delaware,10,Kent County,126704.0,127109.0,405.0,460.0,242.0,218.0,71.0,110.0,181.0,6.0,10001
7,2000,001,Florida,12,Alachua County,217955.0,218602.0,647.0,620.0,382.0,238.0,258.0,138.0,396.0,13.0,12001
8,2000,001,Georgia,13,Appling County,17419.0,17408.0,-11.0,65.0,36.0,29.0,9.0,-49.0,-40.0,0.0,13001
9,2000,001,Hawaii,15,Hawaii County,148676.0,149095.0,419.0,503.0,320.0,183.0,150.0,80.0,230.0,6.0,15001


,year,ESTIMATESBASE,POPESTIMATE,NPOPCHG_,BIRTHS,DEATHS,NATURALINC,INTERNATIONALMIG,DOMESTICMIG,NETMIG,RESIDUAL
count,68002.000000,6.800200e+04,6.800200e+04,68002.000000,68002.000000,68002.000000,68002.000000,68002.000000,68002.000000,68002.000000,68002.000000
mean,2010.500000,1.335707e+04,9.857147e+04,657.678098,1144.683950,744.657495,397.443384,247.213376,8.213126,255.426502,9.483824
std,6.344335,1.215412e+05,3.187636e+05,3720.505443,4170.909892,2137.716754,2232.569346,1691.744043,3630.698286,3172.989624,565.407207
min,2000.000000,0.000000e+00,0.000000e+00,-246640.000000,0.000000,0.000000,-4729.000000,-6061.000000,-248723.000000,-248105.000000,-10353.000000
25%,2005.000000,0.000000e+00,1.101000e+04,-66.000000,100.000000,97.000000,-8.000000,1.000000,-122.000000,-96.000000,-9.000000
50%,2010.500000,0.000000e+00,2.556300e+04,0.000000,262.000000,234.000000,14.000000,7.000000,-12.000000,-2.000000,-1.000000
75%,2016.000000,0.000000e+00,6.580275e+04,223.000000,702.000000,569.000000,134.000000,46.000000,117.000000,161.000000,1.000000
max,2021.000000,1.001401e+07,1.010571e+07,144496.000000,155576.000000,82350.000000,95704.000000,91174.000000,86497.000000,110477.000000,49385.000000


In [5]:
#get curated pat data from multiple csv files
import glob

path = 'C:/Users/TylerFrankenberg/OneDrive/Documents/MScapstone/data/patent/curated/*/*.csv' 
all_files = glob.glob(path)

pat_cols = ['pat_id', 'type', 'pat_number', 'pat_date', 'app_id', 'series_code', 'app_number', 'app_date', 'iloc_county_fips', 'aloc_county_fips',\
             'cpc_group', 'app_year', 'any_ai', 'ml', 'evo', 'nlp', 'speech', 'vision', 'kr', 'planning', 'hardware']
pat_csv_list = []

for file in all_files:
    df = pd.read_csv(file, header=None)
    pat_csv_list.append(df)

pat = pd.concat(pat_csv_list, axis=0, ignore_index=True)
pat.columns = pat_cols

#exclude reissue applications
pat = pat[pat['type'] == 'utility']

#convert FIPS codes to string and add back dropped leading zeroes
pat[['iloc_county_fips', 'aloc_county_fips']] = pat[['iloc_county_fips', 'aloc_county_fips']].fillna('0').astype(float)
pat[['iloc_county_fips', 'aloc_county_fips', ]] = pat[['iloc_county_fips', 'aloc_county_fips', ]].astype(int)
pat[['iloc_county_fips', 'aloc_county_fips']] = pat[['iloc_county_fips', 'aloc_county_fips']].astype(str)
pat.loc[pat['iloc_county_fips'].str.len() == 4, 'iloc_county_fips'] = ['0' + i for i in pat.loc[pat['iloc_county_fips'].str.len() == 4, 'iloc_county_fips']]
pat.loc[pat['aloc_county_fips'].str.len() == 4, 'aloc_county_fips'] = ['0' + i for i in pat.loc[pat['aloc_county_fips'].str.len() == 4, 'aloc_county_fips']]
pat.loc[pat['iloc_county_fips'].str.len() == 1, 'iloc_county_fips'] = None
pat.loc[pat['aloc_county_fips'].str.len() == 1, 'aloc_county_fips'] = None

#add a counter value which will be summed in the grouped dataframe below to give a sum of patents
pat = pat.assign(patents = 1)

C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3777375591.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, header=None)
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3777375591.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, header=None)
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3777375591.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, header=None)
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3777375591.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, header=None)
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3777375591.py:12: DtypeWarning: Columns (10) have mixed types. Spec

In [6]:
#prepare columns from cp dataset to join with patent data
pat_cols = [col for col in cp.columns if col in ['FIPS', 'STNAME', 'CTYNAME', 'year', 'POPESTIMATE', 'DOMESTICMIG']]
pat_pop = cp.loc[:, pat_cols].set_index(['FIPS', 'year'])
pat_pop.head(10)

#group patent dataset by fips and year and join to columns from population dataset
pat_group = pat.groupby(['iloc_county_fips', 'app_year']).agg(sum).reset_index()
pat_group = pat_group.assign(FIPS = pat_group['iloc_county_fips'], year = pat_group['app_year'])
pat_group = pat_group.set_index(['FIPS', 'year'])
pat_group = pat_group.join(pat_pop)

#calculate patents per thousand, ai patents per thousand and domestic net migration as percent of population measures
pat_group = pat_group.assign(pat_per_k = (pat_group['patents'] / pat_group['POPESTIMATE']) * 1000 \
                            ,ai_pat_per_k = (pat_group['any_ai'] / pat_group['POPESTIMATE']) * 1000)

pat_group['SMA_pat_9yr'] = pat_group.groupby('iloc_county_fips')['pat_per_k'].transform(lambda x: x.rolling(9).mean())
pat_group['EMA_pat_9yr'] = pat_group.groupby('iloc_county_fips')['pat_per_k'].transform(lambda x: x.ewm(span=9).mean())
pat_group['SMA_aipat_9yr'] = pat_group.groupby('iloc_county_fips')['any_ai'].transform(lambda x: x.rolling(9).mean())
pat_group['EMA_aipat_9yr'] = pat_group.groupby('iloc_county_fips')['any_ai'].transform(lambda x: x.ewm(span=9).mean())

In [7]:
pat_group = pat_group[['iloc_county_fips', 'STNAME', 'CTYNAME', 'app_year', 'patents', 'any_ai', 'POPESTIMATE', 'DOMESTICMIG', 'pat_per_k', 'ai_pat_per_k', 'EMA_pat_9yr', 'EMA_aipat_9yr']]
print(pat_group.isnull().sum())

pat_group = pat_group.dropna(axis=0, subset=['POPESTIMATE'])
pat_group = pat_group[pat_group['POPESTIMATE'] != 0]

iloc_county_fips       0
STNAME              1161
CTYNAME             1161
app_year               0
patents                0
any_ai                 0
POPESTIMATE         1161
DOMESTICMIG         1161
pat_per_k           1161
ai_pat_per_k        1170
EMA_pat_9yr         1163
EMA_aipat_9yr          0
dtype: int64


In [8]:
display(pat_group.head(10))
display(pat_group.describe())

iloc_county_fips   STNAME         CTYNAME  app_year  patents  \
FIPS  year                                                                
01001 2001            01001  Alabama  Autauga County      2001        1   
      2002            01001  Alabama  Autauga County      2002        2   
      2003            01001  Alabama  Autauga County      2003        1   
      2004            01001  Alabama  Autauga County      2004        1   
      2005            01001  Alabama  Autauga County      2005        2   
      2007            01001  Alabama  Autauga County      2007        3   
      2008            01001  Alabama  Autauga County      2008        3   
      2009            01001  Alabama  Autauga County      2009        1   
      2010            01001  Alabama  Autauga County      2010        1   
      2011            01001  Alabama  Autauga County      2011        2   

            any_ai  POPESTIMATE  DOMESTICMIG  pat_per_k  ai_pat_per_k  \
FIPS  year                                                              
01001 2001     0.0      44434.0        317.0   0.022505           0.0   
      2002     0.0      45157.0        551.0   0.044290           0.0   
      2003     0.0      45762.0        501.0   0.021852           0.0   
      2004     0.0      46933.0        955.0   0.021307           0.0   
      2005     0.0      47870.0        723.0   0.041780           0.0   
      2007     0.0      49834.0        498.0   0.060200           0.0   
      2008     0.0      50354.0        215.0   0.059578           0.0   
      2009     0.0      50756.0        157.0   0.019702           0.0   
      2010     0.0      54773.0        147.0   0.018257           0.0   
      2011     0.0      55227.0        327.0   0.036214           0.0   

            EMA_pat_9yr  EMA_aipat_9yr  
FIPS  year                              
01001 2001     0.022505            0.0  
      2002     0.034608            0.0  
      2003     0.029380            0.0  
      2004     0.026645            0.0  
      2005     0.031147            0.0  
      2007     0.039022            0.0  
      2008     0.044224            0.0  
      2009     0.038331            0.0  
      2010     0.033694            0.0  
      2011     0.034259            0.0

,app_year,patents,any_ai,POPESTIMATE,DOMESTICMIG,pat_per_k,ai_pat_per_k,EMA_pat_9yr,EMA_aipat_9yr
count,43505.000000,43505.000000,43505.000000,4.350500e+04,43505.000000,43505.000000,43505.000000,43505.000000,43505.000000
mean,2011.228411,90.240593,14.715435,1.443037e+05,27.729686,0.332466,0.032596,0.278984,11.562614
std,6.044857,573.846571,148.809565,3.900825e+05,4534.828584,0.632042,0.132057,0.490035,113.824208
min,2000.000000,1.000000,0.000000,4.790000e+02,-248723.000000,0.000406,0.000000,0.000406,0.000000
25%,2006.000000,2.000000,0.000000,2.125800e+04,-163.000000,0.073955,0.000000,0.077941,0.000000
50%,2011.000000,6.000000,0.000000,4.400400e+04,0.000000,0.152317,0.000000,0.140461,0.214758
75%,2016.000000,25.000000,2.000000,1.134090e+05,267.000000,0.342204,0.021991,0.285738,1.319791
max,2021.000000,38681.000000,9745.000000,1.010571e+07,86497.000000,20.514896,5.047659,13.486925,7267.193890


In [9]:
cbsa = pd.read_csv('data/CBSA Crosswalk.csv', header=2)

cbsa = cbsa[(~cbsa['FIPS State Code'].isnull()) & (~cbsa['FIPS County Code'].isnull())]
cbsa[['FIPS State Code', 'FIPS County Code']] = cbsa[['FIPS State Code', 'FIPS County Code']].astype(int).astype('string')
cbsa.loc[cbsa['FIPS State Code'].str.len() == 1, 'FIPS State Code'] = ['0' + i for i in cbsa.loc[cbsa['FIPS State Code'].str.len() == 1, 'FIPS State Code']]
cbsa.loc[cbsa['FIPS County Code'].str.len() == 1, 'FIPS County Code'] = ['00' + i for i in cbsa.loc[cbsa['FIPS County Code'].str.len() == 1, 'FIPS County Code']]
cbsa.loc[cbsa['FIPS County Code'].str.len() == 2, 'FIPS County Code'] = ['0' + i for i in cbsa.loc[cbsa['FIPS County Code'].str.len() == 2, 'FIPS County Code']]
cbsa['FIPS'] = cbsa['FIPS State Code'] + cbsa['FIPS County Code']

cbsa = cbsa.drop(columns=['CBSA Code', 'Metropolitan Division Code', 'CSA Code', 'Metropolitan Division Title', 'FIPS State Code', 'FIPS County Code']).set_index('FIPS')
cbsa = cbsa.rename(columns={'CBSA Title':'cbsa', 'Metropolitan/Micropolitan Statistical Area':'msa', 'CSA Title':'csa', 'County/County Equivalent':'county', 'State Name':'state', 'Central/Outlying County':'central/outlying'})
display(cbsa.head(10))

,cbsa,msa,csa,county,state,central/outlying
FIPS,,,,,,
46013,"Aberdeen, SD",Micropolitan Statistical Area,NaN,Brown County,South Dakota,Central
46045,"Aberdeen, SD",Micropolitan Statistical Area,NaN,Edmunds County,South Dakota,Outlying
53027,"Aberdeen, WA",Micropolitan Statistical Area,NaN,Grays Harbor County,Washington,Central
48059,"Abilene, TX",Metropolitan Statistical Area,NaN,Callahan County,Texas,Outlying
48253,"Abilene, TX",Metropolitan Statistical Area,NaN,Jones County,Texas,Outlying
48441,"Abilene, TX",Metropolitan Statistical Area,NaN,Taylor County,Texas,Central
40123,"Ada, OK",Micropolitan Statistical Area,NaN,Pontotoc County,Oklahoma,Central
26091,"Adrian, MI",Micropolitan Statistical Area,"Detroit-Warren-Ann Arbor, MI",Lenawee County,Michigan,Central
39133,"Akron, OH",Metropolitan Statistical Area,"Cleveland-Akron-Canton, OH",Portage County,Ohio,Central


In [10]:
tax_csv_list = []

for i in list(range(10, 21)):
    df = pd.read_csv(f'data/SOI_tax/{i}incyallagi.csv', header=0, encoding='latin1')
    df['year'] = 2000 + i
    tax_csv_list.append(df)

#for i in tax_csv_list:
#    print(i.head(5))

tax = pd.concat(tax_csv_list, axis=0, ignore_index=True)
tax.head(10)

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,MARS2,PREP,N2,NUMDEP,...,N02910,A02910,N11450,A11450,N10970,A10970,N10971,A10971,N10973,A10973
0,1,AL,0,Alabama,1,55504.0,9767.0,20102.0,63924.0,7506.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AL,0,Alabama,2,933294.0,140316.0,563827.0,1629025.0,517538.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,AL,0,Alabama,3,495863.0,169878.0,303935.0,1081258.0,342804.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,AL,0,Alabama,4,254814.0,156414.0,160348.0,607774.0,187672.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,AL,0,Alabama,5,154603.0,125336.0,94841.0,413668.0,131234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,AL,0,Alabama,6,166872.0,148634.0,100029.0,476977.0,159923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,AL,0,Alabama,7,39324.0,35336.0,30690.0,116722.0,41901.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,AL,1,Autauga County,1,457.0,76.0,149.0,582.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,AL,1,Autauga County,2,10206.0,1330.0,4879.0,15594.0,4884.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,AL,1,Autauga County,3,5736.0,1980.0,2735.0,12246.0,4060.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tax_cols = ['STATEFIPS', 'STATE', 'COUNTYFIPS', 'COUNTYNAME', 'agi_stub', 'N1', 'N2', 'A00100', 'A00700', 'A18425', 'A18450', 'A18500', 'A18800', 'A18460', 'year']
tax_cols_new = ['STATEFIPS', 'STATE', 'COUNTYFIPS', 'COUNTYNAME', 'agi_stub', 'Returns', 'Individuals', 'AGI', 'SL_Refund', 'SL_IncTax', 'SL_SalesTax', 'RE_Tax', 'PropertyTax', 'SL_Lim', 'year']
tax_dict = dict(zip(tax_cols, tax_cols_new))
tax.rename(columns = tax_dict, inplace = True)

tax = tax.loc[(tax['COUNTYNAME'].str.contains('County|Parish|Borough')) & (tax['agi_stub'].isin([7, 8])), [col for col in tax.columns if col in tax_cols_new]]

tax[['STATEFIPS', 'COUNTYFIPS']] = tax[['STATEFIPS', 'COUNTYFIPS']].astype('string')
tax.loc[tax['STATEFIPS'].str.len() == 1, 'STATEFIPS'] = ['0' + i for i in tax.loc[tax['STATEFIPS'].str.len() == 1, 'STATEFIPS']]
tax.loc[tax['COUNTYFIPS'].str.len() == 1, 'COUNTYFIPS'] = ['00' + i for i in tax.loc[tax['COUNTYFIPS'].str.len() == 1, 'COUNTYFIPS']]
tax.loc[tax['COUNTYFIPS'].str.len() == 2, 'COUNTYFIPS'] = ['0' + i for i in tax.loc[tax['COUNTYFIPS'].str.len() == 2, 'COUNTYFIPS']]

tax = tax.assign(FIPS = tax['STATEFIPS'] + tax['COUNTYFIPS'])
tax[tax['year'] == 2020].head(10)

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,Returns,Individuals,AGI,SL_IncTax,SL_SalesTax,RE_Tax,year,SL_Refund,PropertyTax,SL_Lim,FIPS
249021,01,AL,001,Autauga County,7,4080.0,11370.0,547010.0,2833.0,178.0,632.0,2020,498.0,118.0,3544.0,01001
249022,01,AL,001,Autauga County,8,690.0,1950.0,228005.0,3650.0,52.0,519.0,2020,184.0,65.0,2486.0,01001
249029,01,AL,003,Baldwin County,7,16930.0,44530.0,2301144.0,14328.0,1110.0,5305.0,2020,1982.0,674.0,18884.0,01003
249030,01,AL,003,Baldwin County,8,5890.0,16320.0,2597460.0,54956.0,662.0,9958.0,2020,1392.0,807.0,26316.0,01003
249037,01,AL,005,Barbour County,7,750.0,1900.0,99662.0,503.0,58.0,158.0,2020,83.0,33.0,709.0,01005
249038,01,AL,005,Barbour County,8,190.0,460.0,66778.0,840.0,0.0,197.0,2020,31.0,23.0,654.0,01005
249045,01,AL,007,Bibb County,7,850.0,2440.0,111209.0,554.0,0.0,74.0,2020,69.0,25.0,599.0,01007
249046,01,AL,007,Bibb County,8,110.0,300.0,56974.0,1032.0,0.0,107.0,2020,35.0,10.0,480.0,01007
249053,01,AL,009,Blount County,7,2900.0,8270.0,377139.0,1760.0,58.0,352.0,2020,333.0,83.0,1980.0,01009
249054,01,AL,009,Blount County,8,430.0,1150.0,171925.0,3557.0,0.0,448.0,2020,75.0,68.0,1904.0,01009


In [12]:
tax = tax.fillna(0)
print(tax.isnull().sum())
tax = tax.assign(SL_taxtotal = tax['SL_IncTax'] + tax['SL_SalesTax'] + tax['RE_Tax'] + tax['PropertyTax'] + tax['SL_Lim'] - tax['SL_Refund'])
tax = tax.assign(burden = tax['SL_taxtotal'] / tax['AGI'])  
tax.head(10)

STATEFIPS      0
STATE          0
COUNTYFIPS     0
COUNTYNAME     0
agi_stub       0
Returns        0
Individuals    0
AGI            0
SL_IncTax      0
SL_SalesTax    0
RE_Tax         0
year           0
SL_Refund      0
PropertyTax    0
SL_Lim         0
FIPS           0
dtype: int64


,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,Returns,Individuals,AGI,SL_IncTax,SL_SalesTax,RE_Tax,year,SL_Refund,PropertyTax,SL_Lim,FIPS,SL_taxtotal,burden
13,01,AL,001,Autauga County,7,238.0,719.0,90897.0,2454.0,0.0001,362.0,2010,0.0,0.0,0.0,01001,2816.0001,0.030980
20,01,AL,003,Baldwin County,7,2155.0,6328.0,873206.0,27506.0,488.0000,6453.0,2010,0.0,0.0,0.0,01003,34447.0000,0.039449
27,01,AL,005,Barbour County,7,79.0,196.0,36467.0,1041.0,0.0001,268.0,2010,0.0,0.0,0.0,01005,1309.0001,0.035895
34,01,AL,007,Bibb County,7,47.0,119.0,25513.0,691.0,0.0001,73.0,2010,0.0,0.0,0.0,01007,764.0001,0.029946
41,01,AL,009,Blount County,7,172.0,492.0,83339.0,2459.0,0.0001,418.0,2010,0.0,0.0,0.0,01009,2877.0001,0.034522
48,01,AL,011,Bullock County,7,28.0,85.0,11814.0,323.0,0.0001,59.0,2010,0.0,0.0,0.0,01011,382.0001,0.032335
55,01,AL,013,Butler County,7,59.0,148.0,37774.0,1743.0,0.0001,165.0,2010,0.0,0.0,0.0,01013,1908.0001,0.050511
62,01,AL,015,Calhoun County,7,483.0,1397.0,195250.0,5757.0,0.0001,1254.0,2010,0.0,0.0,0.0,01015,7011.0001,0.035908
69,01,AL,017,Chambers County,7,83.0,236.0,31274.0,1027.0,0.0001,252.0,2010,0.0,0.0,0.0,01017,1279.0001,0.040897
76,01,AL,019,Cherokee County,7,64.0,179.0,19916.0,544.0,0.0001,134.0,2010,0.0,0.0,0.0,01019,678.0001,0.034043


In [13]:
#0 = No AGI Stub (Total)
#1 = Under $1
#2 = $1 under $10,000
#3 = $10,000 under $25,000
#4 = $25,000 under $50,000
#5 = $50,000 under $75,000
#6 = $75,000 under $100,000
#7 = $100,000 under $200,000
#8 = $200,000 or more


tax1019 = tax[(tax['year'] >= 2011) & (tax['year'] <= 2019)].groupby(['FIPS', 'agi_stub']).agg(np.mean)

tax1019['burd_weight'] = (tax1019['SL_taxtotal'] / tax1019['AGI']) * tax1019['Returns']
tax1019i = pd.DataFrame(tax1019.groupby('FIPS').agg(sum)['Returns'])
tax1019i = tax1019i.rename(columns={'Returns': 'Returns_Total'})

tax1019 = tax1019.join(tax1019i, on='FIPS')
tax1019['burd_weight'] = tax1019['burd_weight'] / tax1019['Returns_Total']

tax1019a = tax1019.groupby('FIPS').agg(sum)[['Individuals', 'burd_weight']]
display(tax1019.head(10))
display(tax1019i.head(10))
display(tax1019a.head(10))


Returns   Individuals           AGI     SL_IncTax  \
FIPS  agi_stub                                                           
01001 7          2893.888889   8468.888889  3.873709e+05   7056.777778   
      8           435.000000   1265.000000  1.502540e+05   4041.250000   
01003 7         11169.000000  30256.111111  1.575872e+06  30962.666667   
      8          3852.500000  10852.500000  1.640503e+06  48003.000000   
01005 7           594.666667   1540.888889  8.001844e+04   1419.000000   
      8           108.750000    276.250000  4.668575e+04   1151.625000   
01007 7           586.777778   1727.444444  7.738300e+04   1437.111111   
      8            80.000000    208.750000  4.543688e+04   1238.625000   
01009 7          1855.888889   5395.444444  2.444429e+05   4952.777778   
      8           288.750000    805.000000  1.156795e+05   3457.750000   

                SL_SalesTax       RE_Tax    year    SL_Refund  PropertyTax  \
FIPS  agi_stub                                                               
01001 7          336.333344  1413.333333  2015.0  1026.555556   128.333333   
      8           27.625000   538.625000  2015.5   450.500000    43.875000   
01003 7         1728.555556  8633.555556  2015.0  3894.000000   600.111111   
      8          678.000000  8581.125000  2015.5  4588.375000   443.375000   
01005 7          301.111122   394.555556  2015.0   202.111111    39.333333   
      8            0.000000   245.250000  2015.5   115.500000    12.875000   
01007 7           31.555567   191.111111  2015.0   204.555556    23.888889   
      8            0.000000    86.875000  2015.5    82.500000     7.125000   
01009 7          134.000011   883.666667  2015.0   681.666667    96.222222   
      8            0.000000   506.750000  2015.5   383.750000    36.500000   

                     SL_Lim   SL_taxtotal    burden  burd_weight  \
FIPS  agi_stub                                                     
01001 7          827.555556   8735.777789  0.024702     0.019605   
      8          609.625000   4810.500000  0.032060     0.004184   
01003 7         4202.333333  42233.222222  0.028092     0.019927   
      8         6093.500000  59210.625000  0.036195     0.009257   
01005 7          166.222222   2118.111122  0.028020     0.022378   
      8          157.625000   1451.875000  0.032126     0.004808   
01007 7          121.111111   1600.222233  0.023001     0.018198   
      8          111.875000   1362.000000  0.030529     0.003596   
01009 7          480.000000   5865.000011  0.026433     0.020763   
      8          443.750000   4061.000000  0.035244     0.004727   

                Returns_Total  
FIPS  agi_stub                 
01001 7           3328.888889  
      8           3328.888889  
01003 7          15021.500000  
      8          15021.500000  
01005 7            703.416667  
      8            703.416667  
01007 7            666.777778  
      8            666.777778  
01009 7           2144.638889  
      8           2144.638889

,Returns_Total
FIPS,
01001,3328.888889
01003,15021.500000
01005,703.416667
01007,666.777778
01009,2144.638889
01011,153.888889
01013,545.861111
01015,3970.694444
01017,709.388889


,Individuals,burd_weight
FIPS,,
01001,9733.888889,0.023788
01003,41108.611111,0.029183
01005,1817.138889,0.027186
01007,1936.194444,0.021795
01009,6200.444444,0.025489
01011,386.111111,0.028166
01013,1484.166667,0.025062
01015,10830.055556,0.026181
01017,1890.750000,0.028257


In [14]:
edu = pd.read_csv(f'data/edu_attainment/edu_attainment.csv', header=0)
#edu.head(5)

deletecols = list(edu.columns[edu.columns.str.contains('ercent|rban')])
edu = edu.loc[edu['Area name'].str.contains("County|Borough|Parish"), [col for col in edu.columns if col not in deletecols]]
edu_colslist = list(edu.columns[(edu.columns.str.contains("[0-9]{4}")) | \
                                (edu.columns.str.contains("20[0-9]{2}-[0-9]{2}"))])
edu_colslist1 = list(edu.columns[(edu.columns.str.contains("[0-9]{4}")) | \
                                 (edu.columns.str.contains("20[0-9]{2}-[0-9]{2}"))]\
                                    .str.replace("Some college \(1-3 years\)", "Some college or associate's degree")\
                                    .str.replace("Four years of college or higher", "Bachelor's degree or higher")\
                                    .str.replace("2007-11", "2011").str.replace("2016-20", "2020"))

edu_colsdict = dict(zip(edu_colslist, edu_colslist1))
edu.rename(columns=edu_colsdict, inplace=True)
edu.head(10)

stub = list(edu.columns[edu.columns.str.contains('[0-9]{4}')].str.replace('[0-9]{4}.*', '').unique())

edu["id"] = edu.index
edu = pd.wide_to_long(edu, stub, i = "id", j = "year").fillna(0).reset_index()

edu.rename(columns={'Federal Information Processing Standard (FIPS) Code': 'FIPS'}, inplace=True)
edu['FIPS'] = edu['FIPS'].astype(str)
edu.loc[edu['FIPS'].str.len() == 4, 'FIPS'] = ['0' + i for i in edu.loc[edu['FIPS'].str.len() == 4, 'FIPS']]

for i in stub:
    edu.loc[:, i] = edu.loc[:, i].astype(str).str.replace(',', '').astype(int)

edu['year'] = edu['year'].astype(int)
edu = edu[edu['FIPS'] != '15005']

C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3643716205.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace("Some college \(1-3 years\)", "Some college or associate's degree")\
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\3643716205.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  stub = list(edu.columns[edu.columns.str.contains('[0-9]{4}')].str.replace('[0-9]{4}.*', '').unique())


In [15]:
edu_years = [2000, 2011, 2020]
edu_rows = [i for i in cp.index if cp.loc[i, :]['year'] in edu_years]
edu_cols = [col for col in cp.columns if col in ['FIPS', 'year', 'POPESTIMATE']]
edu_pop = cp.loc[edu_rows, edu_cols].set_index(['FIPS', 'year'])
edu_pop.head(10)

edu_group = edu[edu['year'].isin([2011])].groupby(['FIPS', 'year']).agg(sum).reset_index()
edu_group = edu_group.assign(FIPScode = edu_group['FIPS'], YEAR = edu_group['year'])
edu_group = edu_group.set_index(['FIPS', 'year'])
edu_group = edu_group.join(edu_pop)
edu_group = edu_group.assign(bach_pct = edu_group['Bachelor\'s degree or higher, '] / edu_group['POPESTIMATE'])

edu_group['bach_pct'] = edu_group['bach_pct'].fillna(np.mean(edu_group['bach_pct']))

edu_group11 = edu_group[['bach_pct', 'FIPScode', 'YEAR']].rename(columns={'FIPScode':'iloc_county_fips', 'YEAR':'app_year'})
display(edu_group11.head(10))

edu_group11 = pd.pivot_table(edu_group11, index=['iloc_county_fips'], columns=['app_year'], 
                    values=['bach_pct'], aggfunc=np.sum)

edu_group11 = edu_group11.sort_index(axis=1, level=1)
edu_group11.columns = [f'{x}_{y}' for x,y in edu_group11.columns]
edu_group11 = edu_group11.reset_index().set_index('iloc_county_fips')
display(edu_group11.head(10))

,,bach_pct,iloc_county_fips,app_year
FIPS,year,,,
01001,2011,0.135405,01001,2011
01003,2011,0.181547,01003,2011
01005,2011,0.096193,01005,2011
01007,2011,0.065949,01007,2011
01009,2011,0.075521,01009,2011
01011,2011,0.101546,01011,2011
01013,2011,0.083485,01013,2011
01015,2011,0.105016,01015,2011
01017,2011,0.073517,01017,2011


,bach_pct_2011
iloc_county_fips,
01001,0.135405
01003,0.181547
01005,0.096193
01007,0.065949
01009,0.075521
01011,0.101546
01013,0.083485
01015,0.105016
01017,0.073517


In [16]:
hpi = pd.read_csv(f'data/housing/hpi/HPI_AT_BDL_county.csv', header=0)
hpi.head(5)

hpi = hpi[hpi['Year'] >= 2000]
hpi = hpi.loc[:, [col for col in hpi.columns if col not in ['HPI with 1990 base', 'Annual Change (%)']]]

floatcols_hpi = ['HPI', 'HPI with 2000 base']

hpi['HPI'] = hpi['HPI'].str.replace('^.$', '0').astype(float)
hpi['HPI with 2000 base'] = hpi['HPI with 2000 base'].str.replace('^.$', '0').astype(float)
hpi['FIPS'] = hpi['FIPS code'].astype(str)
hpi.loc[hpi['FIPS'].str.len() == 4, 'FIPS'] = ['0' + i for i in hpi.loc[hpi['FIPS'].str.len() == 4, 'FIPS']]

hpi = hpi[hpi['Year'].isin([2011, 2019])]
hpi = pd.pivot_table(hpi, index=['FIPS'], columns=['Year'], 
                    values=['HPI with 2000 base'], aggfunc=np.max)

hpi = hpi.sort_index(axis=1, level=1)
hpi.columns = [f'{x}_{y}' for x,y in hpi.columns]
hpi = hpi.reset_index().set_index('FIPS')
display(hpi.head(10))

mhp = pd.read_csv(f'data/housing/mhp/mhp2011.csv', header=0)
mhp['FIPS'] = mhp['FIPS'].astype(str)
mhp.loc[mhp['FIPS'].str.len() == 4, 'FIPS'] = ['0' + i for i in mhp.loc[mhp['FIPS'].str.len() == 4, 'FIPS']]
mhp = mhp.set_index('FIPS')

mhp.head(10)

hpi = hpi.join(mhp['Estimate Median value (dollars)'])
hpi = hpi.rename(columns={'Estimate Median value (dollars)':'med_home_val_2011', 'HPI with 2000 base_2011':'hpi2011', 'HPI with 2000 base_2019':'hpi2019'})
for col in hpi.columns:
    hpi[[col]] = hpi[[col]].astype(float) 
hpi['med_home_val_2019'] = (hpi['med_home_val_2011'] / hpi['hpi2011']) * hpi['hpi2019']

display(hpi.head(10))

mhv = hpi[['med_home_val_2011', 'med_home_val_2019']]
mhv['pct_chg_mhv'] = (mhv['med_home_val_2019'] - mhv['med_home_val_2011']) / mhv['med_home_val_2011']
mhv = mhv[['med_home_val_2011', 'pct_chg_mhv']]
display(mhv.head(10))

C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\632213873.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  hpi['HPI'] = hpi['HPI'].str.replace('^.$', '0').astype(float)
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\632213873.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  hpi['HPI with 2000 base'] = hpi['HPI with 2000 base'].str.replace('^.$', '0').astype(float)


,HPI with 2000 base_2011,HPI with 2000 base_2019
FIPS,,
01001,125.07,135.85
01003,116.68,166.66
01005,124.41,118.19
01007,127.62,147.46
01009,118.32,136.46
01013,117.47,124.52
01015,123.76,138.32
01017,119.23,135.56
01019,138.58,153.03


,hpi2011,hpi2019,med_home_val_2011,med_home_val_2019
FIPS,,,,
01001,125.07,135.85,137500.0,149351.363237
01003,116.68,166.66,175700.0,250961.278711
01005,124.41,118.19,91600.0,87020.368138
01007,127.62,147.46,87500.0,101102.883561
01009,118.32,136.46,111500.0,128594.405003
01013,117.47,124.52,74300.0,78759.138503
01015,123.76,138.32,99600.0,111317.647059
01017,119.23,135.56,82400.0,93685.683133
01019,138.58,153.03,96200.0,106230.956848


C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\632213873.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mhv['pct_chg_mhv'] = (mhv['med_home_val_2019'] - mhv['med_home_val_2011']) / mhv['med_home_val_2011']


,med_home_val_2011,pct_chg_mhv
FIPS,,
01001,137500.0,0.086192
01003,175700.0,0.428351
01005,91600.0,-0.049996
01007,87500.0,0.155462
01009,111500.0,0.153313
01013,74300.0,0.060015
01015,99600.0,0.117647
01017,82400.0,0.136962
01019,96200.0,0.104272


In [17]:
fmr = pd.read_csv(f'data/housing/fmr/FMR_2Bed_2000_2023.csv', encoding='latin1', header=0)

fmr_cols = list(fmr.columns[fmr.columns.str.contains('_[0-9]')])
fmr_cols_new = list(fmr.columns[fmr.columns.str.contains('_[0-9]')].str.replace('fmr', 'afmr').str.replace('_[0-9].*', ''))
fmr_cols_dict = dict(zip(fmr_cols, fmr_cols_new))
#print(fmr_cols_dict)

fmr.rename(columns=fmr_cols_dict, inplace=True)

stub = list(fmr.columns[fmr.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())
fmr["id"] = fmr.index
fmr = pd.wide_to_long(fmr, stub, i = "id", j = "year").fillna(0).reset_index()

fmr = fmr.loc[:, [col for col in fmr.columns if col not in ['id', 'fips', 'areaname', 'msa', 'fmr']]]

fmr[['state', 'county']] = fmr[['state', 'county']].astype(int).astype('string')
fmr.loc[fmr['state'].str.len() == 1, 'state'] = ['0' + i for i in fmr.loc[fmr['state'].str.len() == 1, 'state']]
fmr.loc[fmr['county'].str.len() == 1, 'county'] = ['00' + i for i in fmr.loc[fmr['county'].str.len() == 1, 'county']]
fmr.loc[fmr['county'].str.len() == 2, 'county'] = ['0' + i for i in fmr.loc[fmr['county'].str.len() == 2, 'county']]

fmr = fmr.assign(FIPS_code = fmr['state'] + fmr['county']).astype(str)
fmr.loc[fmr['year'].str.len() == 1, 'year'] = ['200' + i for i in fmr.loc[fmr['year'].str.len() == 1, 'year']]
fmr.loc[fmr['year'].str.len() == 2, 'year'] = ['20' + i for i in fmr.loc[fmr['year'].str.len() == 2, 'year']]
fmr['year'] = fmr['year'].astype(int)
fmr['afmr'] = fmr['afmr'].astype(float)
fmr.rename(columns={'afmr': 'fmr', 'FIPS_code':'FIPS'}, inplace=True)

fmr = fmr[fmr['year'].isin([2011, 2019])]
fmr = pd.pivot_table(fmr, index=['FIPS'], columns=['year'], 
                    values=['fmr'], aggfunc=np.max)

fmr = fmr.sort_index(axis=1, level=1)
fmr.columns = [f'{x}_{y}' for x,y in fmr.columns]
fmr = fmr.reset_index().set_index('FIPS')
fmr['pct_chg_fmr'] = (fmr['fmr_2019'] - fmr['fmr_2011']) / fmr['fmr_2011']
display(fmr.head(10))


C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\4180848644.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  fmr_cols_new = list(fmr.columns[fmr.columns.str.contains('_[0-9]')].str.replace('fmr', 'afmr').str.replace('_[0-9].*', ''))
C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\4180848644.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  stub = list(fmr.columns[fmr.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())


,fmr_2011,fmr_2019,pct_chg_fmr
FIPS,,,
00000,0.0,1387.0,inf
01001,735.0,825.0,0.122449
01003,764.0,888.0,0.162304
01005,539.0,666.0,0.235622
01007,786.0,873.0,0.110687
01009,786.0,873.0,0.110687
01011,550.0,628.0,0.141818
01013,550.0,628.0,0.141818
01015,585.0,676.0,0.155556


In [18]:
unemp = pd.read_csv(f'data/unemp/Unemployment.csv', header=0)
#unemp.head(5)

unemp = unemp[unemp['Area_name'].str.contains('County|Borough|Parish')]
#unemp.head(5)

print(list(unemp.columns))
unemp = unemp.loc[:, [col for col in unemp.columns if col not in ['Rural_urban_continuum_code_2013', 'Urban_influence_code_2013', 'Metro_2013']]]
stub = list(unemp.columns[unemp.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())
unemp["id"] = unemp.index
unemp = pd.wide_to_long(unemp, stub, i = "id", j = "year").fillna(0).reset_index()

unemp['FIPS_code'] = unemp['FIPS_code'].astype(str)
unemp.loc[unemp['FIPS_code'].str.len() == 4, 'FIPS_code'] = ['0' + i for i in unemp.loc[unemp['FIPS_code'].str.len() == 4, 'FIPS_code']]

unemp['SMA_unemp_9yr'] = unemp.groupby('FIPS_code')['Unemployment_rate_'].transform(lambda x: x.rolling(9).mean())
unemp['EMA_unemp_9yr'] = unemp.groupby('FIPS_code')['Unemployment_rate_'].transform(lambda x: x.ewm(span=9).mean())

unemp = unemp.rename(columns={'FIPS_code':'FIPS'})
unemp = unemp[unemp['year'].isin([2011, 2019])]
unemp = pd.pivot_table(unemp, index=['FIPS'], columns=['year'], 
                    values=['SMA_unemp_9yr', 'EMA_unemp_9yr'], aggfunc=np.max)

unemp = unemp.sort_index(axis=1, level=1)
unemp.columns = [f'{x}_{y}' for x,y in unemp.columns]
unemp['chg_SMA_unemp_9yr'] = (unemp['SMA_unemp_9yr_2019'] - unemp['SMA_unemp_9yr_2011'])
unemp['chg_EMA_unemp_9yr'] = (unemp['EMA_unemp_9yr_2019'] - unemp['EMA_unemp_9yr_2011'])

unemp = unemp.drop(columns=['EMA_unemp_9yr_2019', 'SMA_unemp_9yr_2019'])
display(unemp[unemp.index == '01001'].head(10))

['FIPS_code', 'State', 'Area_name', 'Rural_urban_continuum_code_2013', 'Urban_influence_code_2013', 'Metro_2013', 'Civilian_labor_force_2000', 'Employed_2000', 'Unemployed_2000', 'Unemployment_rate_2000', 'Civilian_labor_force_2001', 'Employed_2001', 'Unemployed_2001', 'Unemployment_rate_2001', 'Civilian_labor_force_2002', 'Employed_2002', 'Unemployed_2002', 'Unemployment_rate_2002', 'Civilian_labor_force_2003', 'Employed_2003', 'Unemployed_2003', 'Unemployment_rate_2003', 'Civilian_labor_force_2004', 'Employed_2004', 'Unemployed_2004', 'Unemployment_rate_2004', 'Civilian_labor_force_2005', 'Employed_2005', 'Unemployed_2005', 'Unemployment_rate_2005', 'Civilian_labor_force_2006', 'Employed_2006', 'Unemployed_2006', 'Unemployment_rate_2006', 'Civilian_labor_force_2007', 'Employed_2007', 'Unemployed_2007', 'Unemployment_rate_2007', 'Civilian_labor_force_2008', 'Employed_2008', 'Unemployed_2008', 'Unemployment_rate_2008', 'Civilian_labor_force_2009', 'Employed_2009', 'Unemployed_2009', 'U

C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_16160\2332635236.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  stub = list(unemp.columns[unemp.columns.str.contains('[0-9]')].str.replace('[0-9].*', '').unique())


,EMA_unemp_9yr_2011,SMA_unemp_9yr_2011,chg_SMA_unemp_9yr,chg_EMA_unemp_9yr
FIPS,,,,
01001,6.533657,5.744444,-0.377778,-1.834208


In [19]:
#get curated natural amenities data from csv
na_cols = ["index", "FIPSCode", "CombinedCountyFIPS", "State", "County", "CensusDiv", "RUC93", "UIC93","MeanTempJan4170", "MeanSunJan4170", "MeanTempJul4170", "MeanRelHumJul4170", "LandSurfTopoCode", "PctH20Area", "NatLogPctH20x100", "JANTEMPZ", "JANSUNZ", "JULTEMPZ", "JULHUMZ", "TOPOGZ", "LNH20AREAZ", "NatAmenScale", "NatAmenRank"]
na_csv_list = []

for i in list(range(0, 4)):
    df = pd.read_csv(f'data/nat_amen/curated/part{i}.csv', header=None)
    na_csv_list.append(df)

#for i in na_csv_list:
#    print(i.head(5))

natamen = pd.concat(na_csv_list, axis=0, ignore_index=True)
natamen.columns = na_cols

natamen[['FIPSCode', 'CombinedCountyFIPS']] = natamen[['FIPSCode', 'CombinedCountyFIPS']].astype('string')
natamen.loc[natamen['FIPSCode'].str.len() == 4, 'FIPSCode'] = ['0' + i for i in natamen.loc[natamen['FIPSCode'].str.len() == 4, 'FIPSCode']]
natamen.loc[natamen['CombinedCountyFIPS'].str.len() == 4, 'CombinedCountyFIPS'] = ['0' + i for i in natamen.loc[natamen['CombinedCountyFIPS'].str.len() == 4, 'CombinedCountyFIPS']]

#print(str(natamen['index'].min()), ' ', str(natamen['CombinedCountyFIPS'].min()))
#print(natamen['index'].max())
#print(natamen['index'].count())

natamen = natamen.rename(columns={'FIPSCode':'FIPS'})
na1999 = natamen[['FIPS', 'NatAmenScale', 'NatAmenRank']].set_index('FIPS')
display(na1999.head(10))

,NatAmenScale,NatAmenRank
FIPS,,
48001,1.07,4
48003,0.68,4
48005,1.92,4
48007,3.71,5
48009,-0.33,3
48011,0.36,4
48013,0.90,4
48015,1.38,4
48017,0.91,4


In [20]:
pat1119 = pat_group[(pat_group['app_year'] == 2011) | (pat_group['app_year'] == 2019)]

pat1119ema = pd.pivot_table(pat1119, index=['iloc_county_fips'], columns=['app_year'], 
                    values=['EMA_pat_9yr', 'EMA_aipat_9yr'], aggfunc=np.max)

pat1119ema = pat1119ema.sort_index(axis=1, level=1)
pat1119ema.columns = [f'{x}_{y}' for x,y in pat1119ema.columns]
pat1119ema = pat1119ema.reset_index().set_index('iloc_county_fips')

pat1119 = pd.pivot_table(pat1119, index=['iloc_county_fips'], columns=['app_year'], 
                    values=['POPESTIMATE', 'patents', 'any_ai'], aggfunc=np.sum)

pat1119 = pat1119.sort_index(axis=1, level=1)
pat1119.columns = [f'{x}_{y}' for x,y in pat1119.columns]
pat1119 = pat1119.reset_index().set_index('iloc_county_fips')
display(pat1119.head(10))
    
pat1119_agg = pat_group[(pat_group['app_year'] >= 2011) & (pat_group['app_year'] <= 2019)]\
    .groupby('iloc_county_fips').agg(sum)
pat1119_agg = pat1119_agg[['patents', 'any_ai', 'DOMESTICMIG']].rename(columns={'patents':'total_patents', 'any_ai':'total_ai_patents', 'DOMESTICMIG':'total_domestic_mig'})
display(pat1119_agg.head(10))

pat1119['pop_chg'] = (pat1119['POPESTIMATE_2019'] - pat1119['POPESTIMATE_2011'])
pat1119['pop_chg_pct'] = (pat1119['POPESTIMATE_2019'] - pat1119['POPESTIMATE_2011']) / pat1119['POPESTIMATE_2011']
pat1119['ppk_2011'] = (pat1119['patents_2011'] / pat1119['POPESTIMATE_2011']) * 1000
pat1119['ppk_2019'] = (pat1119['patents_2019'] / pat1119['POPESTIMATE_2019']) * 1000
pat1119['aippk_2011'] = (pat1119['any_ai_2011'] / pat1119['POPESTIMATE_2011']) * 1000
pat1119['aippk_2019'] = (pat1119['any_ai_2019'] / pat1119['POPESTIMATE_2019']) * 1000

pat1119 = pat1119.join(cbsa).join(pat1119ema).join(pat1119_agg).join(edu_group11).join(tax1019a).join(mhv).join(fmr).join(unemp).join(na1999)

pat1119['EMA_ppk_chg'] = (pat1119['EMA_pat_9yr_2019'] - pat1119['EMA_pat_9yr_2011'])
pat1119['EMA_aippk_chg'] = (pat1119['EMA_aipat_9yr_2019'] - pat1119['EMA_aipat_9yr_2011'])
pat1119['net_pop_chg_domestic_mig'] = pat1119['total_domestic_mig'] / pat1119['POPESTIMATE_2011']
#pat1119['domestic_mig_share'] = pat1119['total_domestic_mig'] / pat1119[pat1119['total_domestic_mig'] >= 0]['total_domestic_mig'].agg('sum')

pat1119['dest_fips'] = pat1119.index
display(pat1119.head(10))


,POPESTIMATE_2011,any_ai_2011,patents_2011,POPESTIMATE_2019,any_ai_2019,patents_2019
iloc_county_fips,,,,,,
01001,55227.0,0.0,2.0,55869.0,1.0,4.0
01003,186558.0,1.0,15.0,223234.0,3.0,38.0
01005,27341.0,0.0,2.0,24686.0,0.0,2.0
01009,57560.0,0.0,3.0,57826.0,0.0,1.0
01015,117744.0,0.0,1.0,113605.0,0.0,5.0
01017,34033.0,0.0,1.0,33254.0,0.0,8.0
01019,25989.0,0.0,3.0,26196.0,0.0,1.0
01021,NaN,NaN,NaN,44428.0,0.0,1.0
01023,13609.0,0.0,1.0,NaN,NaN,NaN


,total_patents,total_ai_patents,total_domestic_mig
iloc_county_fips,,,
01001,17,1.0,727.0
01003,217,11.0,36755.0
01005,16,0.0,-1237.0
01009,17,1.0,11.0
01011,2,0.0,-166.0
01013,3,0.0,-627.0
01015,64,9.0,-4384.0
01017,32,1.0,-488.0
01019,13,0.0,1100.0


,POPESTIMATE_2011,any_ai_2011,patents_2011,POPESTIMATE_2019,any_ai_2019,patents_2019,pop_chg,pop_chg_pct,ppk_2011,ppk_2019,...,EMA_unemp_9yr_2011,SMA_unemp_9yr_2011,chg_SMA_unemp_9yr,chg_EMA_unemp_9yr,NatAmenScale,NatAmenRank,EMA_ppk_chg,EMA_aippk_chg,net_pop_chg_domestic_mig,dest_fips
iloc_county_fips,,,,,,,,,,,,,,,,,,,,,
01001,55227.0,0.0,2.0,55869.0,1.0,4.0,642.0,0.011625,0.036214,0.071596,...,6.533657,5.744444,-0.377778,-1.834208,0.78,4.0,0.005957,0.204607,0.013164,01001
01003,186558.0,1.0,15.0,223234.0,3.0,38.0,36676.0,0.196593,0.080404,0.170225,...,6.834402,5.888889,-0.200000,-1.930724,1.82,4.0,0.058136,0.833397,0.197016,01003
01005,27341.0,0.0,2.0,24686.0,0.0,2.0,-2655.0,-0.097107,0.073150,0.081018,...,9.696023,8.633333,-0.133333,-2.395353,0.19,4.0,0.014270,-0.136776,-0.045243,01005
01009,57560.0,0.0,3.0,57826.0,0.0,1.0,266.0,0.004621,0.052120,0.017293,...,6.671010,5.666667,-0.177778,-1.900473,0.23,4.0,0.006977,0.087965,0.000191,01009
01015,117744.0,0.0,1.0,113605.0,0.0,5.0,-4139.0,-0.035153,0.008493,0.044012,...,7.925555,6.822222,0.211111,-1.865066,0.22,4.0,0.008795,0.565857,-0.037233,01015
01017,34033.0,0.0,1.0,33254.0,0.0,8.0,-779.0,-0.022890,0.029383,0.240573,...,11.420721,10.088889,-3.477778,-5.435407,-0.34,3.0,0.100090,0.169308,-0.014339,01017
01019,25989.0,0.0,3.0,26196.0,0.0,1.0,207.0,0.007965,0.115433,0.038174,...,7.616612,6.566667,-0.833333,-2.623287,0.64,4.0,-0.014877,0.000000,0.042326,01019
01021,NaN,NaN,NaN,44428.0,0.0,1.0,NaN,NaN,NaN,0.022508,...,7.113396,6.088889,-0.388889,-2.153070,0.38,4.0,NaN,NaN,NaN,01021
01023,13609.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.073481,NaN,...,10.259330,9.300000,-0.811111,-2.746646,0.92,4.0,NaN,NaN,-0.025130,01023


In [21]:
pat_scatter = pat1119.drop(columns=['POPESTIMATE_2019', 'pop_chg', 'any_ai_2011', 'any_ai_2019', 'patents_2011', 'patents_2019',\
                                    'ppk_2019', 'aippk_2019', 'total_patents', 'total_ai_patents', 'total_domestic_mig'])


In [22]:
pat_group.to_csv('data/analysis/exploratory/pat_group.csv')
pat1119.to_csv('data/analysis/exploratory/pat1119.csv')
pat_scatter.to_csv('data/analysis/exploratory/pat_scatter.csv')

**County-to-County Inflows**

In [23]:
#get curated ci data from csv
import glob
import os

path = 'C:/Users/TylerFrankenberg/OneDrive/Documents/MScapstone/data/county_migration/curated/*' # use your path
all_files = glob.glob(path)

ci_cols = ['index', 'y2_statefips','y2_countyfips','y1_statefips','y1_countyfips','y1_state','y1_countyname','n1','n2','agi', 'year', 'y2fips', 'y1fips']
ci_csv_list = []

for file in all_files:
    df = pd.read_csv(file, header=None)
    ci_csv_list.append(df)

ci = pd.concat(ci_csv_list, axis=0, ignore_index=True)
ci.columns = ci_cols

ci[['y2fips', 'y1fips']] = ci[['y2fips', 'y1fips']].astype('string')
ci.loc[ci['y2fips'].str.len() == 4, 'y2fips'] = ['0' + i for i in ci.loc[ci['y2fips'].str.len() == 4, 'y2fips']]
ci.loc[ci['y1fips'].str.len() == 4, 'y1fips'] = ['0' + i for i in ci.loc[ci['y1fips'].str.len() == 4, 'y1fips']]

fips_lookup = ci.rename(columns={'y1fips':'fips', 'y1_state':'state', 'y1_countyname':'county'}).loc[:, ['fips', 'state', 'county']]

display(ci.head(20))

,index,y2_statefips,y2_countyfips,y1_statefips,y1_countyfips,y1_state,y1_countyname,n1,n2,agi,year,y2fips,y1fips
0,12,1,1,1,51,AL,Elmore,442,1032,19383,1213,01001,01051
1,13,1,1,1,101,AL,Montgomery,404,877,13828,1213,01001,01101
2,14,1,1,1,21,AL,Chilton,60,146,1955,1213,01001,01021
3,15,1,1,1,47,AL,Dallas,55,125,1560,1213,01001,01047
4,16,1,1,1,73,AL,Jefferson,35,70,1140,1213,01001,01073
5,18,1,1,1,117,AL,Shelby,26,56,1346,1213,01001,01117
6,19,1,1,45,85,SC,Sumter,21,95,552,1213,01001,45085
7,20,1,1,1,97,AL,Mobile,18,39,789,1213,01001,01097
8,21,1,1,1,3,AL,Baldwin,17,46,635,1213,01001,01003
9,22,1,1,1,81,AL,Lee,16,36,638,1213,01001,01081


In [24]:

ci = ci[['year', 'y2fips', 'y1fips', 'n1', 'n2', 'agi']]
ci = ci.rename(columns={'y2fips':'destfips', 'y1fips':'orgfips', 'n1':'returns', 'n2':'indivs'})
print(ci['year'].unique())

[1213 1112  809 1011  910 1617 1718 1920 1516 1314 1819 1415  607  405
  708  506]


In [25]:
ci['year'] = ci['year'].astype(str).str.extract('([0-9][0-9]$)')
ci['year'] = '20' + ci['year']

ci['year'] = ci['year'].astype(int)
print(ci['year'].unique())

[2013 2012 2009 2011 2010 2017 2018 2020 2016 2014 2019 2015 2007 2005
 2008 2006]


In [26]:
print(ci.isnull().sum())
display(ci.head(10))
display(ci.describe())

year        0
destfips    0
orgfips     0
returns     0
indivs      0
agi         0
dtype: int64


,year,destfips,orgfips,returns,indivs,agi
0,2013,01001,01051,442,1032,19383
1,2013,01001,01101,404,877,13828
2,2013,01001,01021,60,146,1955
3,2013,01001,01047,55,125,1560
4,2013,01001,01073,35,70,1140
5,2013,01001,01117,26,56,1346
6,2013,01001,45085,21,95,552
7,2013,01001,01097,18,39,789
8,2013,01001,01003,17,46,635
9,2013,01001,01081,16,36,638


,year,returns,indivs,agi
count,1.087150e+06,1.087150e+06,1.087150e+06,1.087150e+06
mean,2.011537e+03,8.426590e+01,1.556729e+02,4.622993e+03
std,4.447761e+00,3.724922e+02,6.968996e+02,2.422392e+04
min,2.005000e+03,1.000000e+01,1.000000e+01,-1.283400e+06
25%,2.008000e+03,1.600000e+01,2.900000e+01,6.330000e+02
50%,2.011000e+03,2.700000e+01,5.000000e+01,1.256000e+03
75%,2.015000e+03,5.400000e+01,1.020000e+02,2.796000e+03
max,2.020000e+03,2.909200e+04,5.144200e+04,2.252809e+06


In [27]:
ci1119 = ci[(ci['year'] >= 2011) & (ci['year'] <= 2019)].groupby(['destfips', 'orgfips']).agg(sum).reset_index().drop(columns=['year'], axis=1).set_index('orgfips')
ci1119 = ci1119.join(pat1119)

ci1119_cols = [col for col in ci1119.columns if col not in ['destfips', 'dest_fips', 'returns', 'indivs', 'agi']]
y2cols = ['org_' + col for col in ci1119_cols]

ci1119 = ci1119.rename(columns=dict(zip(ci1119_cols, y2cols))).drop(columns=['dest_fips'])
ci1119['orgfips'] = ci1119.index
ci1119['agi'] = (ci1119['agi'] * 1000)
ci1119['hh_agi'] = ci1119['agi'] / ci1119['returns']

ci1119cc = ci1119.set_index('destfips').join(cbsa)
ci1119cc['destfips'] = ci1119cc.index
ci1119cc.loc[ci1119cc['msa'].isnull(), 'msa'] = 'Non MSA'
ci1119cc.loc[ci1119cc['org_msa'].isnull(), 'org_msa'] = 'Non MSA'
ci1119cc.loc[ci1119cc['cbsa'].isnull(), 'cbsa'] = ci1119cc.loc[ci1119cc['cbsa'].isnull(), 'state'] + ' ' + 'Non MSA'
ci1119cc.loc[ci1119cc['org_cbsa'].isnull(), 'org_cbsa'] = ci1119cc.loc[ci1119cc['org_cbsa'].isnull(), 'org_state'] + ' ' + 'Non MSA'
ci1119cc = ci1119cc[ci1119cc['cbsa'] != ci1119cc['org_cbsa']]

co1119cc = ci1119cc.groupby('orgfips')['indivs'].agg(sum)
co1119cc = co1119cc.rename({'indivs':'indivs_out'})

display(ci1119cc.head(10))
display(ci1119cc.columns)

,returns,indivs,agi,org_POPESTIMATE_2011,org_any_ai_2011,org_patents_2011,org_POPESTIMATE_2019,org_any_ai_2019,org_patents_2019,org_pop_chg,...,org_net_pop_chg_domestic_mig,orgfips,hh_agi,cbsa,msa,csa,county,state,central/outlying,destfips
01001,31,67,1132000,186558.0,1.0,15.0,223234.0,3.0,38.0,36676.0,...,0.197016,01003,36516.129032,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,20,54,869000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,01013,43450.000000,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,638,1542,22075000,NaN,NaN,NaN,44428.0,0.0,1.0,NaN,...,NaN,01021,34600.313480,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,527,1216,18578000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,01047,35252.371917,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,11,31,243000,102488.0,0.0,6.0,105882.0,0.0,7.0,3394.0,...,0.018412,01069,22090.909091,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,409,771,16995000,658109.0,22.0,141.0,658573.0,21.0,154.0,464.0,...,-0.029819,01073,41552.567237,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,176,308,7001000,144130.0,4.0,36.0,164542.0,6.0,56.0,20412.0,...,0.084070,01081,39778.409091,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,57,145,2245000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,01085,39385.964912,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,28,46,1129000,339565.0,24.0,184.0,372909.0,45.0,262.0,33344.0,...,0.064671,01089,40321.428571,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001
01001,60,123,2177000,413068.0,1.0,24.0,413210.0,5.0,56.0,142.0,...,-0.038512,01097,36283.333333,"Montgomery, AL",Metropolitan Statistical Area,"Montgomery-Selma-Alexander City, AL",Autauga County,Alabama,Central,01001


Index(['returns', 'indivs', 'agi', 'org_POPESTIMATE_2011', 'org_any_ai_2011',
       'org_patents_2011', 'org_POPESTIMATE_2019', 'org_any_ai_2019',
       'org_patents_2019', 'org_pop_chg', 'org_pop_chg_pct', 'org_ppk_2011',
       'org_ppk_2019', 'org_aippk_2011', 'org_aippk_2019', 'org_cbsa',
       'org_msa', 'org_csa', 'org_county', 'org_state', 'org_central/outlying',
       'org_EMA_aipat_9yr_2011', 'org_EMA_pat_9yr_2011',
       'org_EMA_aipat_9yr_2019', 'org_EMA_pat_9yr_2019', 'org_total_patents',
       'org_total_ai_patents', 'org_total_domestic_mig', 'org_bach_pct_2011',
       'org_Individuals', 'org_burd_weight', 'org_med_home_val_2011',
       'org_pct_chg_mhv', 'org_fmr_2011', 'org_fmr_2019', 'org_pct_chg_fmr',
       'org_EMA_unemp_9yr_2011', 'org_SMA_unemp_9yr_2011',
       'org_chg_SMA_unemp_9yr', 'org_chg_EMA_unemp_9yr', 'org_NatAmenScale',
       'org_NatAmenRank', 'org_EMA_ppk_chg', 'org_EMA_aippk_chg',
       'org_net_pop_chg_domestic_mig', 'orgfips', 'hh_agi', 'c

In [28]:
ci1119cca = ci1119cc[['destfips', 'returns', 'indivs', 'agi']].groupby('destfips').agg(sum)
ci1119cca['hh_agi'] = ci1119cca['agi'] / ci1119cca['returns']
display(ci1119cca.head(10))

ci1119ccb = ci1119cc.drop(columns=['returns', 'indivs', 'agi', 'hh_agi']).groupby('destfips').agg(np.mean)
display(ci1119ccb.head(10))

ci1119ccj = ci1119cca.join(ci1119ccb)
display(ci1119ccj.head(10))

,returns,indivs,agi,hh_agi
destfips,,,,
01001,2610,5789,103183000,39533.716475
01003,28154,55740,1751036000,62194.927897
01005,1838,4346,52220000,28411.316649
01007,1085,2559,44297000,40826.728111
01009,2502,5722,80894000,32331.734612
01011,768,1676,22790000,29674.479167
01013,1107,2548,26253000,23715.447154
01015,10404,21541,364811000,35064.494425
01017,3546,8094,113875000,32113.649182


,org_POPESTIMATE_2011,org_any_ai_2011,org_patents_2011,org_POPESTIMATE_2019,org_any_ai_2019,org_patents_2019,org_pop_chg,org_pop_chg_pct,org_ppk_2011,org_ppk_2019,...,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_SMA_unemp_9yr_2011,org_chg_SMA_unemp_9yr,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_NatAmenRank,org_EMA_ppk_chg,org_EMA_aippk_chg,org_net_pop_chg_domestic_mig
destfips,,,,,,,,,,,,,,,,,,,,,
01001,511739.772727,71.500000,247.954545,573730.590909,124.181818,408.863636,64798.666667,0.099489,0.293278,0.450118,...,0.193788,6.860456,6.000000,-0.347826,-1.953220,1.431739,4.173913,0.120944,44.384474,0.044996
01003,740104.043860,76.403509,385.035088,801756.438596,150.219298,658.149123,63249.054545,0.079012,0.311571,0.471604,...,0.186101,7.188482,6.326554,-0.392185,-1.986860,1.153419,3.940171,0.146200,59.198232,0.026580
01005,526722.000000,14.200000,89.900000,564844.000000,32.300000,168.800000,46015.222222,0.051749,0.111846,0.181810,...,0.207483,7.333125,6.404040,-0.216162,-1.931860,0.203000,3.500000,0.058323,17.155534,-0.012710
01007,106053.000000,2.000000,8.000000,112003.000000,5.000000,26.000000,5950.000000,0.008936,0.070670,0.155930,...,0.173130,8.520847,7.377778,-0.294444,-2.454595,0.965000,4.000000,0.072699,1.273370,-0.019212
01009,148518.875000,4.375000,31.625000,158938.500000,7.250000,51.500000,10419.625000,0.045285,0.136584,0.216948,...,0.152737,7.233862,6.276389,-0.347222,-2.132532,0.580000,3.875000,0.069597,2.457195,0.036112
01011,499916.666667,9.833333,69.666667,530639.333333,30.500000,151.500000,30722.666667,-0.007988,0.112943,0.164614,...,0.192602,7.929687,6.848148,-0.031481,-2.049704,-0.364000,3.200000,0.043706,13.521256,-0.042599
01013,252324.857143,4.285714,33.142857,260446.142857,5.285714,49.428571,8121.285714,0.050614,0.105788,0.171184,...,0.158802,7.057383,6.180952,-0.366667,-1.995858,0.888571,3.714286,0.038158,1.503414,0.021569
01015,360391.942857,50.742857,195.685714,396700.485714,82.885714,299.371429,36241.852941,0.069001,0.275636,0.380794,...,0.197221,7.581958,6.624691,-0.528086,-2.296448,0.636286,3.685714,0.110596,28.909572,0.021369
01017,359392.230769,49.692308,224.461538,419118.583333,91.166667,354.416667,33227.166667,0.072277,0.342456,0.515818,...,0.234555,7.770119,6.929060,-0.822222,-2.415455,0.120769,3.384615,0.162494,32.036745,0.017421


,returns,indivs,agi,hh_agi,org_POPESTIMATE_2011,org_any_ai_2011,org_patents_2011,org_POPESTIMATE_2019,org_any_ai_2019,org_patents_2019,...,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_SMA_unemp_9yr_2011,org_chg_SMA_unemp_9yr,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_NatAmenRank,org_EMA_ppk_chg,org_EMA_aippk_chg,org_net_pop_chg_domestic_mig
destfips,,,,,,,,,,,,,,,,,,,,,
01001,2610,5789,103183000,39533.716475,511739.772727,71.500000,247.954545,573730.590909,124.181818,408.863636,...,0.193788,6.860456,6.000000,-0.347826,-1.953220,1.431739,4.173913,0.120944,44.384474,0.044996
01003,28154,55740,1751036000,62194.927897,740104.043860,76.403509,385.035088,801756.438596,150.219298,658.149123,...,0.186101,7.188482,6.326554,-0.392185,-1.986860,1.153419,3.940171,0.146200,59.198232,0.026580
01005,1838,4346,52220000,28411.316649,526722.000000,14.200000,89.900000,564844.000000,32.300000,168.800000,...,0.207483,7.333125,6.404040,-0.216162,-1.931860,0.203000,3.500000,0.058323,17.155534,-0.012710
01007,1085,2559,44297000,40826.728111,106053.000000,2.000000,8.000000,112003.000000,5.000000,26.000000,...,0.173130,8.520847,7.377778,-0.294444,-2.454595,0.965000,4.000000,0.072699,1.273370,-0.019212
01009,2502,5722,80894000,32331.734612,148518.875000,4.375000,31.625000,158938.500000,7.250000,51.500000,...,0.152737,7.233862,6.276389,-0.347222,-2.132532,0.580000,3.875000,0.069597,2.457195,0.036112
01011,768,1676,22790000,29674.479167,499916.666667,9.833333,69.666667,530639.333333,30.500000,151.500000,...,0.192602,7.929687,6.848148,-0.031481,-2.049704,-0.364000,3.200000,0.043706,13.521256,-0.042599
01013,1107,2548,26253000,23715.447154,252324.857143,4.285714,33.142857,260446.142857,5.285714,49.428571,...,0.158802,7.057383,6.180952,-0.366667,-1.995858,0.888571,3.714286,0.038158,1.503414,0.021569
01015,10404,21541,364811000,35064.494425,360391.942857,50.742857,195.685714,396700.485714,82.885714,299.371429,...,0.197221,7.581958,6.624691,-0.528086,-2.296448,0.636286,3.685714,0.110596,28.909572,0.021369
01017,3546,8094,113875000,32113.649182,359392.230769,49.692308,224.461538,419118.583333,91.166667,354.416667,...,0.234555,7.770119,6.929060,-0.822222,-2.415455,0.120769,3.384615,0.162494,32.036745,0.017421


In [29]:
cicc_scatter = pat_scatter.join(ci1119ccj).join(co1119cc, rsuffix='_out')
cicc_scatter['net_pop_chg_domestic_indivs'] = (cicc_scatter['indivs'] - cicc_scatter['indivs_out'])/ cicc_scatter['Individuals']

cicc_scatter = cicc_scatter.drop(columns=['returns', 'indivs', 'agi', 'pop_chg_pct', 'org_pop_chg_pct', 'org_POPESTIMATE_2019', 'org_pop_chg', 'org_any_ai_2011', 'org_any_ai_2019', 'org_patents_2011', 'org_patents_2019',\
                                    'org_ppk_2019', 'org_aippk_2019', 'org_total_patents', 'org_total_ai_patents', 'org_total_domestic_mig', 'dest_fips', 'cbsa', 'msa', 'csa', 'county', 'state', 'central/outlying'])

display(dict(zip(list(cicc_scatter.columns), list(cicc_scatter.dtypes))))

#check for null values by column
display(cicc_scatter.isnull().sum())
display(cicc_scatter.isin([np.inf]).sum())

#null handling; drop rows with null endogenous variable & origin counterpart. impute means for null & inf otherwise
cicc_scatter = cicc_scatter[(~cicc_scatter['net_pop_chg_domestic_mig'].isnull()) & (~cicc_scatter['org_net_pop_chg_domestic_mig'].isnull())]

for col in list(cicc_scatter.columns):
    cicc_scatter.loc[:, col] = cicc_scatter.loc[:, col].fillna(np.mean(cicc_scatter.loc[:, col]))
    cicc_scatter.loc[cicc_scatter[col] == np.inf, col] = np.mean(cicc_scatter.loc[cicc_scatter[col] != np.inf, col])

cicc_scatter = cicc_scatter.drop(columns=['ppk_2011', 'aippk_2011', 'fmr_2019', 'SMA_unemp_9yr_2011', 'chg_SMA_unemp_9yr', 'NatAmenRank', 'net_pop_chg_domestic_mig', 'org_ppk_2011',\
      'org_aippk_2011', 'org_fmr_2019', 'org_SMA_unemp_9yr_2011', 'org_chg_SMA_unemp_9yr', 'org_NatAmenRank', 'org_net_pop_chg_domestic_mig', 'POPESTIMATE_2011', 'org_POPESTIMATE_2011', 'indivs_out'])

cicc_scatter = cicc_scatter[(cicc_scatter['burd_weight'] < 3) & (cicc_scatter['pct_chg_mhv'] > -1) & (cicc_scatter['net_pop_chg_domestic_indivs'] < 8)]

cicc_scatter['ai_activity_class'] = '0'
cicc_scatter.loc[cicc_scatter['EMA_aipat_9yr_2011'] > 0, 'ai_activity_class'] = '1'

cicc_scatterai = cicc_scatter[cicc_scatter['ai_activity_class'] == '1']

#check for null & inf values by column
display(cicc_scatter.isnull().sum())
display(cicc_scatter.isin([np.inf]).sum())

#view first 10 rows of dataset
display(cicc_scatter.head(10))

#view summary statistics on numerical columns
display(cicc_scatter.describe())
display(cicc_scatterai.describe())

cicc_scatter.to_csv('data/analysis/exploratory/cicc_scatter.csv')

{'POPESTIMATE_2011': dtype('float64'),
 'ppk_2011': dtype('float64'),
 'aippk_2011': dtype('float64'),
 'EMA_aipat_9yr_2011': dtype('float64'),
 'EMA_pat_9yr_2011': dtype('float64'),
 'EMA_aipat_9yr_2019': dtype('float64'),
 'EMA_pat_9yr_2019': dtype('float64'),
 'bach_pct_2011': dtype('float64'),
 'Individuals': dtype('float64'),
 'burd_weight': dtype('float64'),
 'med_home_val_2011': dtype('float64'),
 'pct_chg_mhv': dtype('float64'),
 'fmr_2011': dtype('float64'),
 'fmr_2019': dtype('float64'),
 'pct_chg_fmr': dtype('float64'),
 'EMA_unemp_9yr_2011': dtype('float64'),
 'SMA_unemp_9yr_2011': dtype('float64'),
 'chg_SMA_unemp_9yr': dtype('float64'),
 'chg_EMA_unemp_9yr': dtype('float64'),
 'NatAmenScale': dtype('float64'),
 'NatAmenRank': dtype('float64'),
 'EMA_ppk_chg': dtype('float64'),
 'EMA_aippk_chg': dtype('float64'),
 'net_pop_chg_domestic_mig': dtype('float64'),
 'hh_agi': dtype('float64'),
 'org_POPESTIMATE_2011': dtype('float64'),
 'org_ppk_2011': dtype('float64'),
 'org_ai

POPESTIMATE_2011                360
ppk_2011                        360
aippk_2011                      360
EMA_aipat_9yr_2011              360
EMA_pat_9yr_2011                360
EMA_aipat_9yr_2019              211
EMA_pat_9yr_2019                211
bach_pct_2011                     1
Individuals                       0
burd_weight                       0
med_home_val_2011                90
pct_chg_mhv                     271
fmr_2011                          1
fmr_2019                          1
pct_chg_fmr                       1
EMA_unemp_9yr_2011                0
SMA_unemp_9yr_2011                0
chg_SMA_unemp_9yr                 0
chg_EMA_unemp_9yr                 0
NatAmenScale                     13
NatAmenRank                      13
EMA_ppk_chg                     571
EMA_aippk_chg                   571
net_pop_chg_domestic_mig        360
hh_agi                           15
org_POPESTIMATE_2011             41
org_ppk_2011                     41
org_aippk_2011              

POPESTIMATE_2011                0
ppk_2011                        0
aippk_2011                      0
EMA_aipat_9yr_2011              0
EMA_pat_9yr_2011                0
EMA_aipat_9yr_2019              0
EMA_pat_9yr_2019                0
bach_pct_2011                   1
Individuals                     0
burd_weight                     0
med_home_val_2011               0
pct_chg_mhv                     0
fmr_2011                        0
fmr_2019                        0
pct_chg_fmr                     0
EMA_unemp_9yr_2011              0
SMA_unemp_9yr_2011              0
chg_SMA_unemp_9yr               0
chg_EMA_unemp_9yr               0
NatAmenScale                    0
NatAmenRank                     0
EMA_ppk_chg                     0
EMA_aippk_chg                   0
net_pop_chg_domestic_mig        0
hh_agi                          0
org_POPESTIMATE_2011            0
org_ppk_2011                    0
org_aippk_2011                  0
org_EMA_aipat_9yr_2011          0
org_EMA_pat_9y

EMA_aipat_9yr_2011             0
EMA_pat_9yr_2011               0
EMA_aipat_9yr_2019             0
EMA_pat_9yr_2019               0
bach_pct_2011                  0
Individuals                    0
burd_weight                    0
med_home_val_2011              0
pct_chg_mhv                    0
fmr_2011                       0
pct_chg_fmr                    0
EMA_unemp_9yr_2011             0
chg_EMA_unemp_9yr              0
NatAmenScale                   0
EMA_ppk_chg                    0
EMA_aippk_chg                  0
hh_agi                         0
org_EMA_aipat_9yr_2011         0
org_EMA_pat_9yr_2011           0
org_EMA_aipat_9yr_2019         0
org_EMA_pat_9yr_2019           0
org_bach_pct_2011              0
org_Individuals                0
org_burd_weight                0
org_med_home_val_2011          0
org_pct_chg_mhv                0
org_fmr_2011                   0
org_pct_chg_fmr                0
org_EMA_unemp_9yr_2011         0
org_chg_EMA_unemp_9yr          0
org_NatAme

EMA_aipat_9yr_2011             0
EMA_pat_9yr_2011               0
EMA_aipat_9yr_2019             0
EMA_pat_9yr_2019               0
bach_pct_2011                  0
Individuals                    0
burd_weight                    0
med_home_val_2011              0
pct_chg_mhv                    0
fmr_2011                       0
pct_chg_fmr                    0
EMA_unemp_9yr_2011             0
chg_EMA_unemp_9yr              0
NatAmenScale                   0
EMA_ppk_chg                    0
EMA_aippk_chg                  0
hh_agi                         0
org_EMA_aipat_9yr_2011         0
org_EMA_pat_9yr_2011           0
org_EMA_aipat_9yr_2019         0
org_EMA_pat_9yr_2019           0
org_bach_pct_2011              0
org_Individuals                0
org_burd_weight                0
org_med_home_val_2011          0
org_pct_chg_mhv                0
org_fmr_2011                   0
org_pct_chg_fmr                0
org_EMA_unemp_9yr_2011         0
org_chg_EMA_unemp_9yr          0
org_NatAme

,EMA_aipat_9yr_2011,EMA_pat_9yr_2011,EMA_aipat_9yr_2019,EMA_pat_9yr_2019,bach_pct_2011,Individuals,burd_weight,med_home_val_2011,pct_chg_mhv,fmr_2011,...,org_pct_chg_mhv,org_fmr_2011,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_EMA_ppk_chg,org_EMA_aippk_chg,net_pop_chg_domestic_indivs,ai_activity_class
iloc_county_fips,,,,,,,,,,,,,,,,,,,,,
01001,0.000000,0.034259,0.204607,0.040216,0.135405,9733.888889,0.023788,137500.0,0.086192,735.0,...,0.356271,768.869565,0.193788,6.860456,-1.953220,1.431739,0.120944,44.384474,-0.059072,0
01003,0.700992,0.069517,1.534389,0.127653,0.181547,41108.611111,0.029183,175700.0,0.428351,764.0,...,0.372643,807.762712,0.186101,7.188482,-1.986860,1.153419,0.146200,59.198232,0.470826,1
01005,0.192255,0.084277,0.055479,0.098546,0.096193,1817.138889,0.027186,91600.0,-0.049996,539.0,...,0.225098,707.454545,0.207483,7.333125,-1.931860,0.203000,0.058323,17.155534,-0.198664,1
01009,0.000000,0.027920,0.087965,0.034897,0.075521,6200.444444,0.025489,111500.0,0.153313,786.0,...,0.180477,634.125000,0.152737,7.233862,-2.132532,0.580000,0.069597,2.457195,-0.162730,0
01015,0.322644,0.049145,0.888501,0.057940,0.105016,10830.055556,0.026181,99600.0,0.117647,585.0,...,0.327373,744.138889,0.197221,7.581958,-2.296448,0.636286,0.110596,28.909572,-0.332316,1
01017,0.000000,0.035679,0.169308,0.135770,0.073517,1890.750000,0.028257,82400.0,0.136962,536.0,...,0.294733,739.615385,0.234555,7.770119,-2.415455,0.120769,0.162494,32.036745,0.188814,0
01019,0.000000,0.080444,0.000000,0.065567,0.081034,2212.250000,0.027217,96200.0,0.104272,560.0,...,0.272556,648.777778,0.189026,8.341741,-2.614700,0.045556,0.050173,7.344366,0.334953,0
01023,0.000000,0.088454,26.041298,0.413351,0.075244,1407.805556,0.023082,61400.0,0.281248,536.0,...,0.101777,600.285714,0.213464,9.672941,-2.621140,0.414286,0.017149,0.905671,-0.149168,0
01025,0.000000,0.079595,0.000000,0.049660,0.088717,2735.444444,0.023429,79900.0,0.060566,536.0,...,0.181619,671.300000,0.141778,8.795069,-2.404697,0.590000,0.039273,1.527754,-0.347658,0


,EMA_aipat_9yr_2011,EMA_pat_9yr_2011,EMA_aipat_9yr_2019,EMA_pat_9yr_2019,bach_pct_2011,Individuals,burd_weight,med_home_val_2011,pct_chg_mhv,fmr_2011,...,org_med_home_val_2011,org_pct_chg_mhv,org_fmr_2011,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_EMA_ppk_chg,org_EMA_aippk_chg,net_pop_chg_domestic_indivs
count,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2.031000e+03,2031.000000,2031.000000,2031.000000,2031.000000,...,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000
mean,10.669779,0.266469,25.638799,0.413016,0.140003,3.074246e+04,0.040598,153146.073684,0.282510,719.690236,...,175012.077354,0.349582,789.456940,0.210477,7.078038,-2.028369,0.406200,0.191752,57.174540,0.001333
std,82.663645,0.436657,194.244791,0.653299,0.057830,8.758122e+04,0.016407,89123.432426,0.192986,197.396563,...,64647.716829,0.156305,151.798353,0.052189,1.261555,0.647238,2.027057,0.132189,61.451207,0.293977
min,0.000000,0.001148,0.000000,0.004299,0.041231,6.333333e+01,0.002005,58700.000000,-0.116772,506.000000,...,71800.000000,-0.003848,517.333333,0.017641,3.095940,-4.545130,-4.980000,-0.588468,-0.969337,-2.192970
25%,0.000000,0.079659,0.163311,0.119217,0.099754,2.632375e+03,0.029702,97700.000000,0.155373,595.000000,...,125350.000000,0.237082,669.376923,0.181144,6.462127,-2.403333,-0.950303,0.103571,7.832145,-0.119103
50%,0.271003,0.136927,0.909111,0.231421,0.125875,6.171278e+03,0.039489,129000.000000,0.248644,654.000000,...,157904.081633,0.333412,765.400000,0.207547,7.083443,-1.993950,0.410000,0.179284,44.872865,-0.012359
75%,1.558754,0.279092,9.636121,0.413351,0.167081,1.967006e+04,0.050156,173350.000000,0.343000,783.000000,...,212191.057990,0.433670,882.264170,0.235748,7.834858,-1.646315,1.385444,0.261027,80.998510,0.102314
max,2744.543268,7.948089,6640.517273,11.207404,0.494453,1.801248e+06,0.102816,993900.000000,1.441225,1833.000000,...,493100.000000,1.289500,1461.000000,0.572363,12.129753,0.195093,8.080000,1.695507,498.647757,1.642747


,EMA_aipat_9yr_2011,EMA_pat_9yr_2011,EMA_aipat_9yr_2019,EMA_pat_9yr_2019,bach_pct_2011,Individuals,burd_weight,med_home_val_2011,pct_chg_mhv,fmr_2011,...,org_med_home_val_2011,org_pct_chg_mhv,org_fmr_2011,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_EMA_ppk_chg,org_EMA_aippk_chg,net_pop_chg_domestic_indivs
count,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1.365000e+03,1365.000000,1365.000000,1365.000000,1365.000000,...,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000
mean,15.875693,0.324645,35.020435,0.487863,0.156260,4.396934e+04,0.044471,174027.408737,0.304897,771.430673,...,193050.959033,0.370369,834.608486,0.207505,7.136785,-2.062040,0.654764,0.223541,70.966475,0.031044
std,100.434324,0.468939,236.274625,0.736295,0.060426,1.039584e+05,0.016694,95681.791901,0.212451,213.546087,...,65482.636810,0.151011,149.866688,0.045008,1.041308,0.560495,2.034312,0.131572,63.240630,0.273063
min,0.029366,0.011725,0.005712,0.021555,0.044917,3.669445e+02,0.007914,58800.000000,-0.116772,506.000000,...,71800.000000,-0.003848,536.000000,0.017641,3.183276,-4.243202,-4.532857,-0.418987,-0.529301,-1.433109
25%,0.262295,0.093925,0.426209,0.138037,0.111806,5.199611e+03,0.032858,114100.000000,0.157847,623.000000,...,141385.714286,0.264626,713.923077,0.182536,6.641514,-2.365775,-0.658667,0.140919,24.173650,-0.091976
50%,0.758956,0.170919,1.656514,0.264560,0.144385,1.191528e+04,0.044135,147900.000000,0.256777,707.000000,...,181180.769231,0.364006,821.961538,0.206571,7.093410,-2.004452,0.677879,0.211088,62.399573,0.003957
75%,3.699897,0.358235,10.349686,0.530085,0.187273,3.653044e+04,0.054321,199200.000000,0.395268,851.000000,...,237205.013928,0.448692,928.160714,0.231506,7.760897,-1.735776,1.528072,0.287677,97.125460,0.120728
max,2744.543268,6.486206,6640.517273,11.207404,0.494453,1.801248e+06,0.102816,840900.000000,1.441225,1833.000000,...,493100.000000,1.079248,1461.000000,0.517996,10.713296,-0.172459,8.080000,1.695507,498.647757,1.642747


In [30]:
display(cicc_scatter[cicc_scatter['net_pop_chg_domestic_indivs'] > 8])

,EMA_aipat_9yr_2011,EMA_pat_9yr_2011,EMA_aipat_9yr_2019,EMA_pat_9yr_2019,bach_pct_2011,Individuals,burd_weight,med_home_val_2011,pct_chg_mhv,fmr_2011,...,org_pct_chg_mhv,org_fmr_2011,org_pct_chg_fmr,org_EMA_unemp_9yr_2011,org_chg_EMA_unemp_9yr,org_NatAmenScale,org_EMA_ppk_chg,org_EMA_aippk_chg,net_pop_chg_domestic_indivs,ai_activity_class
iloc_county_fips,,,,,,,,,,,,,,,,,,,,,
